In [1]:
#!pip install pql
import pql

In [2]:
from optimade.filter import Parser
from lark import Transformer
p = Parser(version=(0, 9, 6))

GrammarError: Using an undefined rule: NonTerminal('start')

In [7]:
import pql
from optimade.filter import Parser
from lark import Transformer


optiMadeToPQLOperatorSwitch = {
    "=":"==",
    "<=":"<=",
    ">=":">=",
    "=>":">=",
    "!=": "!=",
    "<":"<",
    ">":">",
}

def OptiMadeToPQLOperatorValidator(x):
    return optiMadeToPQLOperatorSwitch[x]

def combineMultiple(string, index):
    firstQuoteIndex = 0
    for firstQuoteIndex in reversed(range(0, index)):
        if(ord(string[firstQuoteIndex]) == 34):
            break
    lastQuoteIndex = index
    for lastQuoteIndex in range(index, len(string)):
        if(ord(string[lastQuoteIndex])== 34):
            break
    insertion = string[firstQuoteIndex+1:lastQuoteIndex]
    insertion = insertion.split(",")
    return string[:firstQuoteIndex] + 'all({})'.format(insertion) + string[lastQuoteIndex+1:]

def parseInput(PQL):
    length = len(PQL)
    i = 0
    while(i < length):
        if(PQL[i] == ","):
            PQL = combineMultiple(PQL, i)
        i = i + 1 
    return PQL

class OptimadeToPQLTransformer(Transformer):
    def comparison(self, args):
        A = str(args[0])
        B = str(args[2])
        operator = OptiMadeToPQLOperatorValidator(args[1])
#         print('"' + A +'"' + operator + '"' + B + '"')
#         return '"' + A +'"' + operator + '"' + B + '"'
        return A + operator + B
    def atom(self, args):
#         print("atom:", args, len(args))
        return args[0]
    def term(self, args):
#         print("term:", args, len(args))
        result = ""
        for arg in args:
            if arg.lower() == "and" or arg.lower() == "or":
                arg = arg.lower()
            result = result + " " + arg.lower()
        return "(" + result.lstrip() + ")"
    
    def expression(self, args):
#         print("expression:",args)
        result = ""
        for arg in args:
            lower = arg.lower()
            if(lower == "and" or lower == "or"):
                arg = lower
            result = result + " " + arg
        return result.lstrip()
    def start(self, args):
        return args[1]

def optimadeToMongoDBConverter(optimadeQuery):
    p = Parser(version=(0, 9, 5))
    print("optimadeQuery:", optimadeQuery)
    print()
    tree = p.parse(optimadeQuery)
#     print("tree:\n", tree)
#     print()
    rawPQL = OptimadeToPQLTransformer().transform(tree)
    cleanedPQL = parseInput(rawPQL)
    print("cleanedPQL:", cleanedPQL)
    print()
    try:
        return pql.find(cleanedPQL)
    except:
        return "ERROR: Unable to execute conversion"
    

In [8]:
print(optimadeToMongoDBConverter('filter=(A<1 AnD B > 2) or C = 3'))

print(optimadeToMongoDBConverter('filter=A<1 AnD (B > 2 or C = 3)'))

optimadeQuery: filter=(A<1 AnD B > 2) or C = 3

cleanedPQL: ((((a<1) and b>2) or c==3))

{'$or': [{'$and': [{'a': {'$lt': 1}}, {'b': {'$gt': 2}}]}, {'c': 3}]}
optimadeQuery: filter=A<1 AnD (B > 2 or C = 3)

cleanedPQL: (a<1) and (((b>2) or c==3))

{'$and': [{'a': {'$lt': 1}}, {'$or': [{'b': {'$gt': 2}}, {'c': 3}]}]}


# def parseInput(PQL):
    length = len(PQL)
    i = 0
    while(i < length):
        if(PQL[i] == ","):
            PQL = combineMultiple(PQL, i)
        i = i + 1 
    return PQL


rawPQL = 'd<1 and (a<0 or c >2)'
cleanedPQL = parseInput(rawPQL)
pql.find(cleanedPQL)

I am just going to use PQL as my processor with a pre-filter and a post-filter <br>
pre-filter:
1. make sure that Optimade turns into the right PQL format. Note that they are exactly the same except optimade accept = while PQL only accept ==
2. put quote around int s.t. PQL will interpret them as string for now


Post-filter:
1. turn anything that is possibly a floating point turnable into a floating point


In [28]:
rawPQL = 'd<1 and (a<0 or c >2)'


In [ ]:
""

In [ ]:
import unittest
class ConverterTest(unittest.TestCase):
    def setup(self):
        pass
    def testAll(self):
        self.test_one_input()
        self.test_two_inputs_with_and()
        self.test_two_inputs_with_or()
        self.test_valid_numbers_positive()
        
    def test_one_input(self):
        testCase = "filter=a<0" 
        answer = {'a': {'$lt': 0}}
        message = "test_one_input failed"
        self.assertEqual(optimadeToMongoDBConverter(testCase), answer, message)
        print("passed: test_one_input passed")
    def test_two_inputs_with_and(self):
        testCase = "filter=a<0 and b>2" 
        answer = {'$and': [{'a': {'$lt': 0}}, {'b': {'$gt': 2}}]}
        message = "test_two_inputs_with_and failed"
        self.assertEqual(optimadeToMongoDBConverter(testCase), answer, message)
        print("passed: test_two_inputs_with_and")
    def test_two_inputs_with_or(self):
        testCase = "filter=a<0 or b>2" 
        answer = {'$or': [{'a': {'$lt': 0}}, {'b': {'$gt': 2}}]}
        message = "test_two_inputs_with_or failed"
        self.assertEqual(optimadeToMongoDBConverter(testCase), answer, message)
        print("passed: test_two_inputs_with_or")
    def test_valid_numbers_with_positive_sign(self):
        testCase = "filter= a=+1" 
        answer = {'a': +1}
        message = "test_valid_numbers_positive failed"
        self.assertEqual(optimadeToMongoDBConverter(testCase), answer, message)
        print("passed: test_valid_numbers_positive")

In [ ]:
ConverterTest().testAll()

In [2]:
import pql
from optimade.filter import Parser
from lark import Transformer

p = Parser(version=(0, 9, 5))
x = p.parse('filter= A<1 and b < 2')
print(x.pretty())

start
  filter=
  expression
    term
      term
        atom
          comparison
            A
            <
            1
      and
      atom
        comparison
          b
          <
          2



In [1]:
import pql
from optimade.filter import Parser
from lark import Transformer

p = Parser(version=(0, 9, 5))
print(p.parse('filter= (A<1 or b < 2) and c < 1').pretty())

start
  filter=
  expression
    term
      term
        term
          term
            atom
              comparison
                A
                <
                1
          or
          atom
            comparison
              b
              <
              2
        and
        atom
          comparison
            c
            <
            1



In [2]:
print(p.parse('filter= A<1 or (b < 2 and c < 1)').pretty())

start
  filter=
  expression
    expression
      term
        atom
          comparison
            A
            <
            1
    or
    term
      term
        term
          atom
            comparison
              b
              <
              2
        and
        atom
          comparison
            c
            <
            1



In [3]:
p.parse('filter= A<1 or (b < 2 and c < 1)') == p.parse('filter= (A<1 or b < 2) and c < 1')

False